<a href="https://colab.research.google.com/github/leetm219/Crosswalk/blob/main/our_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 두 가지 카테고리 분류
# zzllrr imagergeek 다운로드
# 수집하고 싶은 이미지 검색
# 우측 상단 아이콘 클릭 -> Go 버튼 누른후 화면 전환
# 필요 없는 이미지, 아이콘 파일 클릭해서 제거
# rename 버튼 클릭후 prefix버튼 클릭 이름 명명 규칙 수정
# 수정한 이미지 이름 preview 버튼 클릭후 반영 확인
# save 누른 후 수집 대기 -> 수집 완료 후
# 해당 카테고리 폴더 생성후 모두 저장
# 상위 폴더 생성후 카테고리 폴더를 모두 넣기
# 상위 폴더 압축 -> our_dataset.zip
# carload and crosswalk
# 최근 여러 나라에서는 횡단보도 앞에서 일단 정지한다.
# 하지만 한국에서는 빨강불에만 정지한다. 이에 발생하는 사고를 방지하기위해 횡단보도를 찾는 모델을 개발한다.

# 목표 성능 90%
# 현재 89.8% > HT
# 현재 85%
# 1. DATA
# 2. Model 교체 혹은 확장
# 3. HT


In [2]:
# # drive에 압축파일 업로드 하고
# # 아래 코드 실행
# !unzip /content/drive/MyDrive/our_dataset.zip

Archive:  /content/drive/MyDrive/our_dataset.zip
replace carload/carload-001.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# 파일을 접근 -> 메모리에 파일을 올려야 -> 모든 파일의 경로가 필요
from glob import glob
# mother_path = '/content/chexpert/train'
# dataset 폴더가 없는 상태에서 풀렸다면
mother_path = '/content/'
glob(mother_path + '/*/*')

In [4]:
import os
a = '/content'
b = 'test'
os.path.join(a,b) # = a + '/' + b 리눅스, 맥, window

'/content/test'

In [5]:
from PIL import Image
import numpy as np

In [ ]:
folder_list = ['carload', 'crosswalk']
train = []
label = []
for idx, folder_name in enumerate(folder_list):
    folder = os.path.join(mother_path, folder_name)
    all_files_path = glob(folder + '/*')
    for k, file_path in enumerate(all_files_path):
        # 파일 불러오기
        img = Image.open(file_path)
        # 이미지 파일 전처리
        img = img.resize((128,128), Image.BILINEAR)
        img = img.convert('L')
        np_img = np.array(img)
        train.append(np_img)
        label.append(idx)
        if k % 10 == 0:
            print(f'{k}/{len(all_files_path)}')
np_train = np.array(train)
np_train = np.expand_dims(np_train, axis=-1)
np_label = np.array(label)

In [29]:
print(np.bincount(np_label))
print(np_train.shape)
# np.save()

[ 501 1582]
(2083, 128, 128, 1)


In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np_train, np_label, test_size=0.2)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(1666, 128, 128, 1) (417, 128, 128, 1) (1666,) (417,)


In [9]:
# xy = (x_train,x_test,y_train,y_test)
# np.save('preprocessed.npy',xy)
# np.load('preprocessed.npy')

In [31]:
x_train = x_train / 255.
x_test = x_test / 255.

In [32]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [33]:
input_layer = layers.Input(shape=(128, 128, 1), name='start layer')
tower_1_1 = layers.Conv2D(64, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name='tower_1_1')(input_layer)
tower_1_2_1= layers.Conv2D(64, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name='tower_1_2_1')(input_layer)
tower_1_2_2= layers.Conv2D(64, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu', name = 'tower_1_2_2')(tower_1_2_1)
tower_1_3_1= layers.Conv2D(64, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name = 'tower_1_3_1')(input_layer)
tower_1_3_2= layers.Conv2D(64, kernel_size=(5,5), padding='same', strides=(1,1), activation='relu', name = 'tower_1_3_2')(tower_1_3_1)
tower_1_4_1= layers.MaxPool2D(pool_size=(3,3), strides=(1,1), padding='same', name = 'tower_1_4_1')(input_layer)
tower_1_4_2= layers.Conv2D(64, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name = 'tower_1_4_2')(tower_1_4_1)
concat_layer_1 = layers.concatenate([tower_1_1,tower_1_2_2,tower_1_3_2,tower_1_4_2],axis=3)
concat_layer_1 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(concat_layer_1)

tower_2_1 = layers.Conv2D(120, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name='tower_2_1')(concat_layer_1)
tower_2_2_1= layers.Conv2D(120, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name='tower_2_2_1')(concat_layer_1)
tower_2_2_2= layers.Conv2D(120, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu', name = 'tower_2_2_2')(tower_2_2_1)
tower_2_3_1= layers.Conv2D(120, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name = 'tower_2_3_1')(concat_layer_1)
tower_2_3_2= layers.Conv2D(120, kernel_size=(5,5), padding='same', strides=(1,1), activation='relu', name = 'tower_2_3_2')(tower_2_3_1)
tower_2_4_1= layers.MaxPool2D(pool_size=(3,3), strides=(1,1), padding='same', name = 'tower_2_4_1')(concat_layer_1)
tower_2_4_2= layers.Conv2D(120, kernel_size=(1,1), padding='same', strides=(1,1), activation='relu', name = 'tower_2_4_2')(tower_2_4_1)
concat_layer_2 = layers.concatenate([tower_2_1,tower_2_2_2,tower_2_3_2,tower_2_4_2],axis=3)
concat_layer_2 = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer_2)
last_avg_pool = layers.AveragePooling2D(pool_size=(31,31) , strides=(1,1), padding='valid')(concat_layer_2)
flat_layaer = layers.Flatten()(last_avg_pool)
output_layer = layers.Dense(1, activation='sigmoid')(flat_layaer)

# 변경 포인트 input, lastavg_pool,output_layer
model = models.Model(input_layer,output_layer)
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start layer (InputLayer)    [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 tower_1_2_1 (Conv2D)        (None, 128, 128, 64)         128       ['start layer[0][0]']         
                                                                                                  
 tower_1_3_1 (Conv2D)        (None, 128, 128, 64)         128       ['start layer[0][0]']         
                                                                                                  
 tower_1_4_1 (MaxPooling2D)  (None, 128, 128, 1)          0         ['start layer[0][0]']         
                                                                                            

In [22]:
!mkdir tensorboard_log

mkdir: cannot create directory ‘tensorboard_log’: File exists


In [34]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
tb = tf.keras.callbacks.TensorBoard(log_dir='/content/tensorboad_log', histogram_freq=1)

model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=10, callbacks=[tb])

Epoch 1/10
53/53 [==============================] - 17s 250ms/step - loss: 0.5444 - accuracy: 0.7545 - val_loss: 0.5586 - val_accuracy: 0.7506
Epoch 2/10
53/53 [==============================] - 13s 252ms/step - loss: 0.5337 - accuracy: 0.7617 - val_loss: 0.5916 - val_accuracy: 0.7506
Epoch 3/10
53/53 [==============================] - 13s 242ms/step - loss: 0.5374 - accuracy: 0.7617 - val_loss: 0.5517 - val_accuracy: 0.7506
Epoch 4/10
53/53 [==============================] - 13s 249ms/step - loss: 0.5263 - accuracy: 0.7617 - val_loss: 0.5554 - val_accuracy: 0.7506
Epoch 5/10
53/53 [==============================] - 13s 240ms/step - loss: 0.5194 - accuracy: 0.7617 - val_loss: 0.6815 - val_accuracy: 0.7506
Epoch 6/10
53/53 [==============================] - 13s 247ms/step - loss: 0.5245 - accuracy: 0.7617 - val_loss: 0.5338 - val_accuracy: 0.7506
Epoch 7/10
53/53 [==============================] - 13s 247ms/step - loss: 0.5169 - accuracy: 0.7617 - val_loss: 0.5270 - val_accuracy: 0.7506

In [40]:
from google.colab import files
f = files.upload()

Saving cross.jpg to cross.jpg


In [41]:
test_file_name = list(f.keys())[0]

In [42]:
img = Image.open(test_file_name)
img = img.resize((128,128), Image.BILINEAR)
img = img.convert('L')
data = np.asarray(img)
data = np.expand_dims(data, axis=-1)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor/255.
test_image_tensor.shape

(1, 128, 128, 1)

In [43]:
result = model.predict(test_image_tensor)
if result[0][0] > 0.5:
    print(f'{folder_list[0]} 일 확률이 {result[0][0] * 100} % 입니다')
else:
    print(f'{folder_list[1]} 일 확률이 {(1-result[0][0]) * 100} % 입니다')

1/1 [==============================] - 0s 18ms/step
carload 일 확률이 66.76780581474304 % 입니다
